In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hlsf.models import LSF_DATA
from scipy import interpolate
from astropy.io import fits
from astropy import wcs
%matplotlib qt

In [2]:
config = 'HK'
lamp = 'Kr'

In [3]:
lsf_data = LSF_DATA(f"../exposures/ARC-{lamp}_CLEAR_20MAS_{config}_PRM.fits", f"../text/{lamp}.txt", f"../exposures/WAVECAL_TABLE_20MAS_{config}.fits", 
                        f"../exposures/SLITLET_TABLE_20MAS_{config}.fits", slice=9, normal='Normal')
lsf_data.get_line_list()

{170: 14104.298,
 173: 14517.84,
 184: 15474.026,
 185: 15681.02,
 186: 15820.09,
 187: 16726.513,
 188: 16785.128,
 193: 17098.771,
 196: 17616.854,
 197: 17842.737,
 198: 18002.229,
 199: 18167.315,
 200: 18399.786,
 201: 18580.896,
 202: 18696.294,
 205: 20209.878,
 208: 21165.471,
 209: 21902.513,
 210: 22485.775,
 211: 23340.416}

In [15]:
hdul = fits.open('../images/image_HK.fits')
image = hdul['INTENSITY'].data

In [16]:
import warnings
with warnings.catch_warnings():
    # Ignore a warning on using DATE-OBS in place of MJD-OBS
    warnings.filterwarnings('ignore', message="'datfix' made the change",
                            category=wcs.FITSFixedWarning)
    wcs = wcs.WCS(hdul['INTENSITY'].header)

In [18]:
fig = plt.figure()
fig.add_subplot(111, projection=wcs)
plt.imshow(image, origin='lower', cmap=plt.cm.viridis)
plt.show()